# Inventário

Deve-se executar as seis primeiras células para carregar funções utilizadas para rodar o script

In [1]:
import json
import os
from openpyxl import load_workbook
from datetime import timedelta
import random
from copy import deepcopy
from django.contrib.gis.geos import LineString, Point
from django.contrib.admin.utils import flatten
import math
import datetime
import locale
from tqdm.notebook import tqdm
from PIL import Image
from io import BytesIO
from simple_history.utils import bulk_create_with_history
from django.contrib.gis.geos import Point, LineString as DjangoLineString
from helpers.histories import bulk_update_with_history
import pytz

In [2]:
from helpers.route_maker import dic_to_ordered_list, unequal_point_pairs

In [3]:
from django.contrib.gis.geos import Point, LineString as DjangoLineString
from django.contrib.gis.db.models.functions import Distance
from turfpy.misc import nearest_point_on_line
from turfpy.measurement import length
from geojson import LineString, loads
from helpers.histories import bulk_update_with_history
from tqdm.notebook import tqdm

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [4]:
def km_to_coordinates(road, km):

    road_marks = dic_to_ordered_list(road.marks)
    selected_pair = None

    for pair in unequal_point_pairs(road_marks):
        if pair[0]["km"] > km and pair[1]["km"] < km:
            selected_pair = pair
            break
        elif pair[1]["km"] > km and pair[0]["km"] < km:
            selected_pair = pair
            break
        elif pair[0]["km"] == km:
            return Point(pair[0]["point"]["coordinates"]), road
        elif pair[1]["km"] == km:
            return Point(pair[1]["point"]["coordinates"]), road

    # Calculate total length
    start_km = min([selected_pair[0]["km"], selected_pair[1]["km"]])
    end_km = max([selected_pair[0]["km"], selected_pair[1]["km"]])
    segment_mark_length = end_km - start_km
    distance_from_min = km - start_km
    # Check if km is decreasing
    invert_km = selected_pair[1]["km"] <= selected_pair[0]["km"]

    # Cut segment
    start_key = min((selected_pair[0]["index"], selected_pair[1]["index"]))
    end_key = max((selected_pair[0]["index"], selected_pair[1]["index"]))

    points = []
    for key in range(start_key, end_key + 1):
        points.append(
            Point(road.path[key][0], road.path[key][1], road.path[key][2])
        )

    segment = LineString(points, srid=4326)
    # Apply km inversion correction
    if invert_km:
        segment_distance = (
            1 - (distance_from_min / segment_mark_length)
        ) * segment.length
    else:
        segment_distance = (
            distance_from_min / segment_mark_length
        ) * segment.length
    # Find point
    point = segment.interpolate(segment_distance)

    return Point(point.x, point.y), road

In [5]:
def check_valid_road(road, km):

    road_marks = dic_to_ordered_list(road.marks)

    for pair in unequal_point_pairs(road_marks):
        if pair[0]["km"] >= km and pair[1]["km"] <= km:
            return True
        elif pair[1]["km"] >= km and pair[0]["km"] <= km:
            return True

    return False

In [6]:
def get_road_coordinates(road_name, km, direction, company):
    road_set = Road.objects.filter(
        name__contains=road_name, direction=int(direction), company=company
    )

    # If roads in specified direction are not found, search road only
    # by name and order then by direction
    road_set_generic = Road.objects.filter(
        name__contains=road_name, company=company
    ).order_by("direction")

    # Check if KM range in road_set
#     print("Searching KM on direction...")
    valid = False
    for road in road_set:
        if check_valid_road(road, km):
#             print("Found KM on Road {}".format(road))
            valid = True
            break

    if not valid:
#         print("Searching KM without direction...")
        for road in road_set_generic:
            if check_valid_road(road, km):
#                 print("Found KM on Road {}".format(road))
                valid = True
                break

    if not valid:
        return Point(0, 0), None

    try:
        return km_to_coordinates(road, km)
    except Exception:
        return Point(0, 0), None

In [7]:
def findIndex(flist, func):
    for i,v in enumerate(flist):
        if func(v): 
            return i
    return -1

In [8]:
def point_to_km(lng, lat, road):
    point = Point(lng, lat, srid=4326)
#     roads = Road.objects.filter(company=company).annotate(distance=Distance(point, 'path')).order_by('distance')
#     road = roads[0]
    path = loads(road.path.geojson)
    point_geojson = loads(point.geojson)
    road_marks = list(road.marks.values())
    snap_point = nearest_point_on_line(path, point_geojson)
    

    road_markers = sorted([a for a in road_marks if 'index' in a or ('index' in a and a['index'] == 0)], key=lambda x: int(x['key']))
#     road_markers = sorted([a for a in road_marks if a['index'] or a['index'] == 0], key=lambda x: int(x['key']))
    rm_index = findIndex(road_markers, lambda x: x['index'] > snap_point['properties']['index'])
    
    if (rm_index == -1):
        rm_index = len(road_markers) - 1;
    
    km_marker_start = road_markers[rm_index - 1]
    km_marker_end = road_markers[rm_index]
    
    if km_marker_start['index'] == road_markers[-1]['index']: 
        snap = road_markers[-1]
        km = snap['km']
    elif km_marker_end['index'] == road_markers[0]['index']:
        snap = road_markers[0]
        km = snap['km']
    else:
        test = path['coordinates'][km_marker_start['index']:km_marker_end['index'] + 1]
        marker_path = LineString(test)
        snap = nearest_point_on_line(marker_path, point_geojson)        
        ls_length = length(marker_path, units="km")
        km = ((snap['properties']['location'] / ls_length)        
            * (km_marker_end['km'] - km_marker_start['km'])
            + km_marker_start['km'])
    
    return road, road.name, point, round(km, 3)

In [9]:
from collections import defaultdict
from django.db.models.signals import *


class DisableSignals(object):
    def __init__(self, disabled_signals=None):
        self.stashed_signals = defaultdict(list)
        self.disabled_signals = disabled_signals or [
            pre_init, post_init,
            pre_save, post_save,
            pre_delete, post_delete,
            pre_migrate, post_migrate,
        ]

    def __enter__(self):
        for signal in self.disabled_signals:
            self.disconnect(signal)

    def __exit__(self, exc_type, exc_val, exc_tb):
        for signal in list(self.stashed_signals):
            self.reconnect(signal)

    def disconnect(self, signal):
        self.stashed_signals[signal] = signal.receivers
        signal.receivers = []

    def reconnect(self, signal):
        signal.receivers = self.stashed_signals.get(signal, [])
        del self.stashed_signals[signal]

In [10]:
def get_connected_reference(company, resource, key, value):
    select_options = company.custom_options[resource]['fields'][key]['select_options']['options']
    return next(a['value'] for a in select_options if a['name'].lower() == value.lower())

In [11]:
def get_value(field, occ, value):
    values = occ.form_fields['fields']
    item_translation = flatten([item['selectOptions']['options'] for item in values if item['displayName']==field])
#     print(item_translation)

    final_translation = {item['name']: item['value'] for item in item_translation}
#     print(final_translation)
    return final_translation[value]

 Verificando se está no ambiente de produção

In [12]:
!cat .env

STAGE=PRODUCTION


Escolhendo nome do arquivo Excel para carregar o Inventário

In [13]:
filename="Cópia de Descida D'água - ERS-446 OK"

Escolhendo a aba da planilha do Excel

In [14]:
wb = load_workbook(filename + '.xlsx')
sheetname = wb.sheetnames[1]
ws = wb[sheetname]

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


Testando planilha carregada do Excel

In [15]:
print(ws['A1'].value)

km


Carregando os valores no script

In [16]:
header = []
values = []

for index, row in enumerate(ws.rows):
    if index == 0:
        header = list([a.value for a in row])
        continue
    obj = {}
    for col_index, cell in enumerate(row):
        value = cell.value
        obj[header[col_index]] = value
    if obj['Rodovia'] is not None:           
   
        obj['img'] = [a for a in ws._images if a.anchor._from.row == index]

        try:
            for a in obj['img']:
                a.name = 'CSG'
#             obj['img'][0].name='Verso'
        except:
            pass
        
    values.append(obj)

Teste de valores importados da planilha

In [17]:
len(values),values[0]

(198,
 {'km': None,
  'km final': None,
  'km de Projeto': None,
  'km final de Projeto': None,
  'Latitude_Inicial': -29.368777777777776,
  'Longitude_Inicial': -51.375175,
  'Latitude_Final': None,
  'Longitude_Final': None,
  'Status': 'Identificado',
  'Equipe/Empreiteira': 'Cadastro incial',
  'Encontrado em': None,
  'Executado em': None,
  'Sentido': 'Não se Aplica',
  'Classe': "Descida D'água",
  'Faixa': 'Não se aplica',
  'Rodovia': 'ERS-446',
  'Tipo': 'DAR',
  'Largura\n(m)': 0.5,
  'Extensão\n(m)': 2,
  'Lado montante': 'Direita',
  'Dissipador': 'NÃO',
  'Estado de conservação': 'Bom',
  'Observações': None,
  'Foto_1': None,
  'Data Foto_1': None,
  'Tipo Foto_1': None,
  'Descrição Foto_1': None,
  'Foto_2': None,
  'Data Foto_2': None,
  'Tipo Foto_2': None,
  'Descrição Foto_2': None,
  'Foto_3': None,
  'Data Foto_3': None,
  'Tipo Foto_3': None,
  'Descrição Foto_3': None,
  'Foto_4': None,
  'Data Foto_4': None,
  'Tipo Foto_4': None,
  'Descrição Foto_4': None,
 

Definindo valores da importação

In [18]:
company = Company.objects.get(name='Caminhos da Serra Gaúcha')

#Classe Descida D'agua
occurrence_type=OccurrenceType.objects.get(uuid='05c0b8e0-bc5d-4702-8464-676e45298fdc')

user=User.objects.get(username='rlcs')
status=ServiceOrderActionStatus.objects.get(companies=company, name='Identificado')
# step=ApprovalStep.objects.get(approval_flow__company=company, name='Em Elaboração')
firm=Firm.objects.get(uuid='0edd2ecc-8d5e-4444-898b-0069a3e9c354')
sign_name='Retrorrefletância'
company,occurrence_type,firm

(<Company: 4edb7778-e350-4e77-8e1e-de5f87b1da7f: Caminhos da Serra Gaúcha>,
 <OccurrenceType: Descida D'água - ['Caminhos da Serra Gaúcha']>,
 <Firm: [4edb7778-e350-4e77-8e1e-de5f87b1da7f: Caminhos da Serra Gaúcha] 0edd2ecc-8d5e-4444-898b-0069a3e9c354: Cadastro incial>)

In [20]:
road=Road.objects.get(company=company,name__icontains="446")
road

<Road: 2061: ERS-446>

In [ ]:
for a in values:
    a['Lado montante'] = 'D'

In [21]:
values[0]['Latitude_Inicial'],values[0]['Longitude_Inicial']
road,_,point,km =point_to_km(float(values[0]['Longitude_Inicial']),float(values[0]['Latitude_Inicial']),road)
road,point,km,values[0]

(<Road: 2061: ERS-446>,
 0.881,
 {'km': None,
  'km final': None,
  'km de Projeto': None,
  'km final de Projeto': None,
  'Latitude_Inicial': -29.368777777777776,
  'Longitude_Inicial': -51.375175,
  'Latitude_Final': None,
  'Longitude_Final': None,
  'Status': 'Identificado',
  'Equipe/Empreiteira': 'Cadastro incial',
  'Encontrado em': None,
  'Executado em': None,
  'Sentido': 'Não se Aplica',
  'Classe': "Descida D'água",
  'Faixa': 'Não se aplica',
  'Rodovia': 'ERS-446',
  'Tipo': 'DAR',
  'Largura\n(m)': 0.5,
  'Extensão\n(m)': 2,
  'Lado montante': 'Direita',
  'Dissipador': 'NÃO',
  'Estado de conservação': 'Bom',
  'Observações': None,
  'Foto_1': None,
  'Data Foto_1': None,
  'Tipo Foto_1': None,
  'Descrição Foto_1': None,
  'Foto_2': None,
  'Data Foto_2': None,
  'Tipo Foto_2': None,
  'Descrição Foto_2': None,
  'Foto_3': None,
  'Data Foto_3': None,
  'Tipo Foto_3': None,
  'Descrição Foto_3': None,
  'Foto_4': None,
  'Data Foto_4': None,
  'Tipo Foto_4': None,
  '

Criando itens de Inventário

In [24]:
objects=[]


for index, a in enumerate(tqdm(values)):

    long=str(a['Longitude_Inicial'])
    lat=str(a['Latitude_Inicial'])
#     pista = 'Pista Dupla' if a['Pista'] == 'DUPLA' else 'Pista Simples'
    road,_,point,km =point_to_km(float(long),float(lat),road)
    if '446' not in road.name:
        print(f"Rodovia {road} - Km:{km} - Linha: {index+2}")
    try:
        
        objects.append((Reporting(
                    company=company,
                    occurrence_type=occurrence_type,
                    lane=str(get_connected_reference(company, 'reporting', 'lane', 'Não se Aplica')),
                    road=road,
                    road_name=road.name,
                    direction=str(get_connected_reference(company, 'reporting', 'direction', a['Sentido'])),
                    created_by=user,
#                     found_at=datetime.datetime('2023-05-08'),
                    km=float(km),
#                     end_km=float(end_km) if a['Longitude_Final'] is not None and a['Latitude_Final'] else None,
                    point=point,
                    firm=firm,
                    status=status,
                    form_data={
                        'type': a['Tipo'],
                        'side': get_value('Lado montante',occurrence_type,a['Lado montante']) if a["Lado montante"] is not '' else None,
                        'dissipador': True if str(a['Dissipador']).lower() == 'sim' else False,
                        'state': get_value('Estado de conservação',occurrence_type,a["Estado de conservação"].capitalize()) if a["Estado de conservação"] is not None else None,
                        'width': a['Largura\n(m)'] if a['Largura\n(m)'] is not None else None,
                        'extension': a['Extensão\n(m)'] if a['Extensão\n(m)'] is not None else None,
                        
                    }, 
                    ),a['img']))
    except Exception as e:

        print(f'Arquivo"{filename}" - Rodovia {road} - Km:{km} - Linha: {index+2} - {e}')
        print(objects.form_data)

  0%|          | 0/198 [00:00<?, ?it/s]

Arquivo"Cópia de Descida D'água - ERS-446 OK" - Rodovia 2061: ERS-446 - Km:0.881 - Linha: 2 - 'Direita'


AttributeError: 'list' object has no attribute 'form_data'

Salvando itens de Inventário no sistema e imprimindo os seriais

In [26]:
len(objects),objects[0]

(28,
 (<Reporting: [Caminhos da Serra Gaúcha]  - 2023-06-22 13:09:57.739357+00:00>,
  [<openpyxl.drawing.image.Image at 0x7f23e75c4710>]))

In [27]:
for reporting, images in tqdm(objects):
    reporting.save()
    print(reporting.number)

  0%|          | 0/28 [00:00<?, ?it/s]

Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2023.05876
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2023.05877
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2023.05878
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2023.05879
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2023.05880
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2023.05881
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2023.05882
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2023.05883
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CS

Anexando as imagens

In [28]:
for reporting, images in tqdm(objects):
    for image in reversed(images):
        reporting_file = ReportingFile(
            created_by=user,
            reporting=reporting,
            description=image.name,
            km=reporting.km,
            point=reporting.point
        )
        reporting_file.save()
        image_io = BytesIO(image._data())
        im = Image.open(image_io)
        if im.mode in ("RGBA", "P"):
            im = im.convert("RGB")
        thumb_io = BytesIO()
        im.save(thumb_io, format='jpeg', quality=90)
        reporting_file.upload.save(image.name + '.jpeg', thumb_io)

  0%|          | 0/28 [00:00<?, ?it/s]